In [1]:
#Librerías
import ee #earthengine
import geemap #Alternativa a GEE paquete
import folium #Mapas dinámicos
from folium import plugins #https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=P-tesHgSk5Se
import netCDF4 as nc #Lectura de archivos .nc
import xarray #convertir los archivos de imagen en un arreglo
import matplotlib.pyplot as plt #Generación de gráficos
import pandas as pd
import numpy as np
import os

In [2]:
geemap.update_package() #Actualizar geemap

Unzipping geemap-master.zip ...
Data downloaded to: C:\Users\valiz\Downloads\geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


In [3]:
import geemap

In [4]:
Map = geemap.Map() #Cargar geemap

In [5]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
Incendios = ee.ImageCollection('FIRMS')
Temperatura = "T21"
Confianza = "confidence"
banda = Confianza

In [7]:
#Crear mascara
mascara = ee.Geometry.Rectangle([-75.1999998,  11.8999998, -73.2000017,   8.3000000])

In [8]:
Incendios_Filtro = Incendios.select(banda).filterDate('2017-01-01','2022-06-01').filterBounds(mascara)

In [9]:
# Función para cortar catalogo de imagenes FIRMS
def cortarImagenes(img):
    imgCortada = img.clip(mascara).copyProperties(img)
    return imgCortada

In [10]:
IncendiosClipped = Incendios_Filtro.map(cortarImagenes)

In [11]:
#Quitar mascaras al imgCol
def UnMasking(img):
    #multiply = img.multiply(0.1)
    #divide = multiply.divide(10)
    #reprojected = divide.unitScale(0, 1).reproject('EPSG:4326', None, 1000)
    #UnM = reprojected.unmask(0)
    #UnM2 = UnM.multiply(100)
    
    UnMasking = img.unmask(0) #0 for temperature and -1 for confidense
    UnMasking2 = UnMasking.reproject('EPSG:4326', None, 1000).copyProperties(img)
    
    return UnMasking2.copyProperties(img)

In [12]:
IncendiosClipped = IncendiosClipped.map(UnMasking)

In [13]:
#IncendiosClipped.select(banda).getInfo()

In [14]:
Map.centerObject(mascara,10)

In [15]:
#Añadir capa FIRMS
vis_param_FIRE = {'min': 0, 'max': 500, 'palette':['black','blue','green','yellow','red']}
Map.addLayer(IncendiosClipped.first(), {}, 'FIRES')
#Map.addLayer(mascara, {}, 'mask')

In [16]:
#Función para reducir la resolución de las imagenes FIRMS
"""
def reducirResolucion(img):
    #ee.Image(img)
    #img= img.select(banda)
    #return img.reduceResolution(reducer= ee.Reducer.max(), maxPixels= 1024).reproject(crs='EPSG:4326',scale= 44399.95765686035).copyProperties(img)
    Reduction = img.reduceResolution(**{'reducer': ee.Reducer.max(),'maxPixels': 1024}).reproject(**{'crs': 'EPSG:4326','scale': 44399.95765686035})
    return Reduction
"""

"\ndef reducirResolucion(img):\n    #ee.Image(img)\n    #img= img.select(banda)\n    #return img.reduceResolution(reducer= ee.Reducer.max(), maxPixels= 1024).reproject(crs='EPSG:4326',scale= 44399.95765686035).copyProperties(img)\n    Reduction = img.reduceResolution(**{'reducer': ee.Reducer.max(),'maxPixels': 1024}).reproject(**{'crs': 'EPSG:4326','scale': 44399.95765686035})\n    return Reduction\n"

In [17]:
"""
FIRMSReducido = IncendiosClipped.map(reducirResolucion)
"""

'\nFIRMSReducido = IncendiosClipped.map(reducirResolucion)\n'

In [18]:
#FIRMSReducido.select(banda).getInfo()

In [19]:
#Añadir capa FIRMS
"""
Map.addLayer(FIRMSReducido.max(), vis_param_FIRE, 'FIRES_REDUCED')
"""

"\nMap.addLayer(FIRMSReducido.max(), vis_param_FIRE, 'FIRES_REDUCED')\n"

In [20]:
#Para la temperatura
# To reduced maps use scale 44399.95765686035
#ExpImgs = geemap.ee_export_image_collection(IncendiosClipped.select(banda), '/FIRMS_TEMP_MG',scale=1000, region=mascara)

In [21]:
#Para la confianza
# To reduced maps use scale 44399.95765686035
ExpImgs2 = geemap.ee_export_image_collection(IncendiosClipped.select(banda), '/FIRMS_CONF_MG',scale=1000, region=mascara)

Total number of images: 1966

Exporting 1/1966: 2017001.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017001.tif


Exporting 2/1966: 2017002.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017002.tif


Exporting 3/1966: 2017003.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017003.tif


Exporting 4/1966: 2017004.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017004.tif


Exporting 5/1966: 2017005.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017005.tif


Exporting 6/1966: 2017006.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017006.tif


Exporting 7/1966: 2017

Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017025.tif


Exporting 26/1966: 2017026.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017026.tif


Exporting 27/1966: 2017027.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017027.tif


Exporting 28/1966: 2017028.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017028.tif


Exporting 29/1966: 2017029.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017029.tif


Exporting 30/1966: 2017030.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017030.tif


Exporting 31/1966: 2017031.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SER

Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017049.tif


Exporting 50/1966: 2017050.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017050.tif


Exporting 51/1966: 2017051.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017051.tif


Exporting 52/1966: 2017052.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017052.tif


Exporting 53/1966: 2017053.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017053.tif


Exporting 54/1966: 2017054.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017054.tif


Exporting 55/1966: 2017055.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Inc

Exporting 74/1966: 2017074.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017074.tif


Exporting 75/1966: 2017075.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017075.tif


Exporting 76/1966: 2017076.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017076.tif


Exporting 77/1966: 2017077.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017077.tif


Exporting 78/1966: 2017078.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017078.tif


Exporting 79/1966: 2017079.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017079.tif


Exporting 80/1966: 2017080.tif
Generating URL 

Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017107.tif


Exporting 99/1966: 2017108.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017108.tif


Exporting 100/1966: 2017109.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017109.tif


Exporting 101/1966: 2017110.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017110.tif


Exporting 102/1966: 2017111.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017111.tif


Exporting 103/1966: 2017112.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2017112.tif


Exporting 104/1966: 2017113.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLED

...

Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022035.tif


Exporting 1851/1966: 2022036.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022036.tif


Exporting 1852/1966: 2022037.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022037.tif


Exporting 1853/1966: 2022038.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022038.tif


Exporting 1854/1966: 2022039.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022039.tif


Exporting 1855/1966: 2022040.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022040.tif


Exporting 1856/1966: 2022041.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_

Exporting 1875/1966: 2022060.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022060.tif


Exporting 1876/1966: 2022061.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022061.tif


Exporting 1877/1966: 2022062.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022062.tif


Exporting 1878/1966: 2022063.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022063.tif


Exporting 1879/1966: 2022064.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022064.tif


Exporting 1880/1966: 2022065.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022065.tif


Exporting 1881/1966: 2022066.tif
G

Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022084.tif


Exporting 1900/1966: 2022085.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022085.tif


Exporting 1901/1966: 2022086.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022086.tif


Exporting 1902/1966: 2022087.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022087.tif


Exporting 1903/1966: 2022088.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022088.tif


Exporting 1904/1966: 2022089.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022089.tif


Exporting 1905/1966: 2022090.tif
Generating URL ...
Please wait ...
Data downloaded to

Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022108.tif


Exporting 1924/1966: 2022109.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022109.tif


Exporting 1925/1966: 2022110.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022110.tif


Exporting 1926/1966: 2022111.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022111.tif


Exporting 1927/1966: 2022112.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022112.tif


Exporting 1928/1966: 2022113.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022113.tif


Exporting 1929/1966: 2022114.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_

Exporting 1948/1966: 2022133.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022133.tif


Exporting 1949/1966: 2022134.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022134.tif


Exporting 1950/1966: 2022135.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022135.tif


Exporting 1951/1966: 2022136.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022136.tif


Exporting 1952/1966: 2022137.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022137.tif


Exporting 1953/1966: 2022138.tif
Generating URL ...
Please wait ...
Data downloaded to D:\DOCS\SERGIO_ARBOLEDA\Incendios\CodigosIpyN\FIRMS_CONF_MG\2022138.tif


Exporting 1954/1966: 2022139.tif
G